In [5]:
# step 1: import needed packages
import pandas as pd
import itertools
import numpy as np

In [6]:
# step 2 data aggregation analysis

In [ ]:

# === Instellingen ===HIER AANPASSEN!!!!!!!!
excel_path = "/Users/biekeleenknegt/Desktop/Real Demandan.xlsx"
sheet_name = "channel_data"  # of pas her aan naar channel_data/size_data/product_data
item_researched= "channel"# pas dit aan naar channel/ size/ product

# === Data inladen ===
df = pd.read_excel(excel_path, sheet_name=sheet_name)
df.set_index(df.columns[0], inplace=True)  # zet 'Rijlabels' als index
df = df.apply(pd.to_numeric, errors='coerce')  # zorg dat alles numeriek is

# === Combinaties maken ===
tobecombined = df.index.tolist()
columns = df.columns.tolist()
combinations = list(itertools.combinations(tobecombined, 2))

# === Bereken MPE ===
results = []

for factor1, factor2 in combinations:
    diffs = (df.loc[factor1] - df.loc[factor2]) / df.loc[factor2]

    row = {
        'Percetage difference': f'{factor1} vs {factor2}',
        **diffs.to_dict(),
        
    }
    results.append(row)

# === Dataframe maken ===
result_df = pd.DataFrame(results)
result_df.set_index('Percetage difference', inplace=True)

# === Naar Excel schrijven (optioneel) ===
result_df.to_excel(f"{item_researched}_mpe_analysis.xlsx")

# === Output tonen ===
print(f"{item_researched} MPE Analysis")
print(result_df)
# === Stap 5: Zet resultaat in DataFrame en schrijf weg naar Excel ===
result_df = pd.DataFrame(results)
result_df.set_index('Percetage difference', inplace=True)

output_path = f"/Users/biekeleenknegt/Documents/Github/SCM_GW/{item_researched}_mpe_analysis.xlsx"
result_df.to_excel(output_path)

print(f"Results saved: {output_path}")


channel MPE Analysis
                             2019      2020      2021      2022      2023  \
Percetage difference                                                        
Amsterdam vs Berlin     -0.106087 -0.009740 -0.017323 -0.050143  0.030429   
Amsterdam vs Brussels   -0.368550 -0.239401 -0.295711 -0.280130 -0.327010   
Amsterdam vs Copenhagen -0.625910 -0.568294 -0.532234 -0.526090 -0.500000   
Amsterdam vs Helsinki   -0.565878 -0.540317 -0.582609 -0.562953 -0.508575   
Amsterdam vs Madrid     -0.399533 -0.406615 -0.312775 -0.365550 -0.233539   
Amsterdam vs Paris      -0.260432 -0.109489 -0.048780 -0.238806 -0.198062   
Amsterdam vs Platform    0.250608  0.389522  0.686486  0.857143  1.104520   
Amsterdam vs Rome        0.552870  1.140351  0.890909  1.033742  0.886076   
Amsterdam vs Stockholm  -0.600622 -0.546805 -0.531532 -0.600121 -0.550392   
Amsterdam vs Webshop    -0.796274 -0.765204 -0.783859 -0.774643 -0.730366   
Berlin vs Brussels      -0.293612 -0.231920 -0.283296 -

In [8]:
import pandas as pd

# === Pad en sheetnaam ===
file_path = f"/Users/biekeleenknegt/Documents/Github/SCM_GW/{item_researched}_mpe_analysis.xlsx"
sheet_name = "Sheet1"

# === Inlezen en voorbereiden ===
df = pd.read_excel(file_path, sheet_name=sheet_name)
df.columns = df.columns.astype(str).str.strip()
df.set_index(df.columns[0], inplace=True)

# === Groepdefinities ===
groepen = {
    'Years': ['2019', '2020', '2021', '2022', '2023'],
    'Countries': ['Amsterdam', 'Berlin', 'Brussels', 'Copenhagen', 'Helsinki',
                  'Madrid', 'Paris', 'Platform', 'Rome', 'Stockholm', 'Webshop'],
    'Products': ['06GE2U92FA', 'DTPQ67872X', 'G8WP29EHC6', 'HYT3MYM7CY', 'IMFA21HLV3',
                 'JU9OS20S99', 'PPB56V08LB', 'R5X6KPETN3', 'RX38XS00QN'],
    'Sizes': ['L', 'M', 'onesize', 'S', 'XL', 'XS', 'XXL', 'XXS']
}

# === Hulpfunctie om blokken te bouwen ===
def maak_blok(df, kolomnamen, label):
    aanwezige = [col for col in kolomnamen if col in df.columns]
    if not aanwezige:
        print(f"❌ Kolommen voor blok '{label}' niet gevonden in de data.")
        return None
    blok = df[aanwezige].copy()
    blok[f"{label} - Average"] = blok.mean(axis=1)
    blok[f"{label} - StdDev"] = blok.std(axis=1)
    return blok

# === Alle blokken verzamelen ===
blokken = []
for naam, kolommen in groepen.items():
    blok = maak_blok(df, kolommen, naam)
    if blok is not None:
        blokken.append(blok)

# === Combineer alle blokken naast elkaar ===
samengesteld_df = pd.concat(blokken, axis=1)
samengesteld_df.insert(0, "Percentage difference --> MPE (mean percentage error)", samengesteld_df.index)

# === Wegschrijven naar Excel ===
output_path = f"/Users/biekeleenknegt/Documents/Github/SCM_GW/mpe_gecombineerd_met_{item_researched}.xlsx"
samengesteld_df.to_excel(output_path, index=False)

print(f"✅ Alles opgeslagen in: {output_path}")




❌ Kolommen voor blok 'Countries' niet gevonden in de data.
✅ Alles opgeslagen in: /Users/biekeleenknegt/Documents/Github/SCM_GW/mpe_gecombineerd_met_channel.xlsx
